In [ ]:
%run funcs.ipynb
%run Proj_org.ipynb

# Classification

In [ ]:
dt = DecisionTreeClassifier()

In [ ]:
dt.fit(X_train, y_train)

In [ ]:
dt.score(X_train, y_train)

In [ ]:
svm =SVC()

In [ ]:
svm.fit(X_train, y_train)

In [ ]:
svm.score(X_train, y_train)

In [ ]:
svm.score(X_test, y_test)

In [ ]:
knn = KNeighborsClassifier()

In [ ]:
knn.fit(X_train, y_train)

In [ ]:
knn.score(X_train, y_train)

In [ ]:
knn.score(X_test, y_test)

In [ ]:
mlpc = MLPClassifier((128 , 64, 32, 16, 32, 470))

In [ ]:
mlpc.fit(X_train, y_train)

In [ ]:
mlpc.score(X_train, y_train)

In [ ]:
mlpc.score(X_test, y_test)

# Regression

In [ ]:
dtR = DecisionTreeRegressor()

In [ ]:
dtR.fit(X_train_r, y_train_r)

In [ ]:
dtR.score(X_train_r, y_train_r)

In [ ]:
dtR.score(X_test_r, y_test_r)

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(X_train_r, y_train_r)

In [ ]:
lr.score(X_train_r, y_train_r)

In [ ]:
pf = PolynomialFeatures(10)

In [ ]:
pf.fit_transform(X_train_r)

In [ ]:
lr.fit(pf.fit_transform(X_train), y_train)

In [ ]:
lr.score(pf.fit_transform(X_train), y_train)

In [ ]:
lr.predict(pf.fit_transform(X_train)).size

In [ ]:
svr = SVR()

In [ ]:
svr.fit(X_train_r, y_train_r)

In [ ]:
svr.score(X_train_r, y_train_r)

In [ ]:
mlpr = MLPRegressor((200, 100, 50, 10, 50, 100, 200))

In [ ]:
mlpr.fit(X_train_r, y_train_r)

In [ ]:
mlpr.score(X_train_r, y_train_r)

# Clustering

In [ ]:
km = KMeans()

In [ ]:
km.fit(X_train)

In [ ]:
km.inertia_

In [ ]:
lofm = LocalOutlierFactor()
lof = LocalOutlierFactor(novelty=True)

In [ ]:
lof.fit(X_train)

In [ ]:
preds = lof.predict(X_train)

In [ ]:
pd.Series(preds).value_counts()/len(preds)

In [ ]:
lof.score_samples(X_test)

# Plots

In [ ]:
plt.figure(dpi=150, figsize=(20,20))
plot_tree(dt)
plt.show()

# Deep Learning with Keras

In [ ]:
model = keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
model.add(layers.Embedding(input_dim=100, output_dim=64))

# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(128))

# Add a Dense layer with 10 units.
model.add(layers.Dense(10))